<a href="https://colab.research.google.com/github/DongDong-Zoez/pytorchAI/blob/main/FSL/Few_shot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Few Shot Learning

在進入 FSL 之前，我們先來看看一個問題

![](https://i.ytimg.com/vi/hE7eGew4eeg/maxresdefault.jpg)

The image copied from [here](https://i.ytimg.com/vi/hE7eGew4eeg/maxresdefault.jpg)

我們有一張 query 影像，我們想知道 query 影像與下列影像中哪個相似度最高，這個任務怎麼達成呢?

最值觀的想法是如果訓練集中有跟 query 影像同類別的樣本的話，我們可以訓練一個 CNN 分類器幫我們判斷，但是這樣做的話有一些必要條件

1. 訓練集中必須包含與 query 影像同類別的其他影像
2. CNN 的訓練需要"足夠多"的影像做訓練

Few shot (小樣本) learning 的提出就是為了解決上述的問題

## 樣本不夠多了?

回憶我們在影像分類任務中，我們要讓模型學會的是抓到影像的"特徵"，再做判斷，但是當樣本不夠多時，我們無法得到一個表現好的分類器，這時我們必須改變我們的目標，即讓模型學會判斷"事物的異同"

- 分類: 提取特徵 $\to$ 判斷哪些類別較可能有這些特徵
- FSL: 提取特徵 $\to$ 逐一比對各個類別的特徵與輸入影像的特徵是否有異同

## Terminology

在 FSL 中，我們會將訓練集稱為 support set，把測試集稱為 query set。通常在 support set 中每個類別會有共同數量的影像，我們會把一個有 $k$ 個類別，且每類別有 $n$ 張影像的任務稱為 $k-$way $n-$shot learning

FSL 已經有很多衍生方法，這章節將介紹其中一個較為熱門的方法 Siamese 網路

## Siamese Network

孿生網路可以說是度量學習 (metric learning) 的一個分枝，度量學習主要判斷兩物體到底相不相似，這與我們過往機器學習的方法不同 (分類、回歸、分群、降維)，這章節我們將介紹最簡易的孿生網路

### 孿生?

注意到我們的目標，是要能讓模型學習到影像之間的距離要怎麼定義，所以我們會有兩個影像作為輸入，輸出則是這兩張影像的距離

假設我們有影像 A、B

- 若 A、B 相似，那麼我們希望模型輸出的值越小越好
- 若 A、B 不相似，那麼我們希望模型輸出的值越大越好

我們希望模型是"公平"看待這兩張影像的，所以我們會把 A、B 放到同一個 CNN 網路提取特徵，這也是我們為什麼稱它為"孿生"，其實這個詞用的不好，會誤導人們以為有兩個網路，事實上在訓練以及推斷過程中都只會有一個網路

![](https://929687.smushcdn.com/2633864/wp-content/uploads/2020/11/keras_siamese_networks_process.png?lossy=1&strip=1&webp=1)

## 訓練方法

孿生網路主要有兩種訓練方法

第一種訓練方法:

我們會輸入兩張影像

- 兩張影像同類別 $\to$ $\mbox{label} = 0$
- 兩張影像不同類別 $\to$ $\mbox{label} = 1$

再用 Contrastive Loss 

$$
\mbox{ContrastiveLoss}(x^1, x^2) = (1-\mbox{label})\frac{1}{2}d_{12}+\mbox{label}\frac{1}{2}\{\max(0, m-d_{12})\}^2
$$

第二種訓練方法:

我們會先選定一張影像作為 anchor $(x^a)$，接著從與 $x^a$ 同一類中選出一張影像作為 positive sample $x^+$，再從與 $x^a$ 不同類中選出一張影像做為 negative sample $x^-$

我們希望在特徵空間上 anchor 與 positive sample 盡量靠近，anchor 與 negative sample 的距離要足夠顯著，也就是

$$
d^+ = ||f(x^a)-f(x^+)||^2 \\
d^- = ||f(x^a)-f(x^-)||^2
$$

- $f:$ embedding fucntion，可以選用 CNN 網路
- $d^+:$ positive sample 與 anchor 的距離，且應盡量小
- $d^-:$ negative sample 與 anchor 的距離，且應盡量大

我們會定義一個臨界值 $\alpha$，並採用 Triplet Loss 

$$
\mbox{TripletLoss}(x^a, x^+, x^-) = \begin{cases}
0&, \mbox{if} \quad  d^- > d^+ + \alpha, \\ 
d^+ + \alpha - d^-&, \mbox{otherwise}
\end{cases} = \max\{0, d^+ + \alpha - d^-\}
$$

Note: 因為在 FSL 中影像過少，所以模型初期很難抓到影像特徵，故我們常會用預訓練和微調方法



### FaceNet

![](https://m-shaeri.ir/blog/wp-content/uploads/2021/04/facenet_cnn.jpg)

TripletLoss 一個很簡單的應用是 FaceNet，可以應用於門禁系統或者配對問題

### Matching Networks 2016

![](https://user-images.githubusercontent.com/23639048/56020148-28af9900-5d0f-11e9-8a2d-c463ea46d3d5.png)

將 query 影像放入 CNN 做 embedding，之後與 support set 中的影像計算 cosine similarity 與 softmax

### Prototypical Networks 2017

注意到在 Matching Networks 中，一個 query 影像要和全部的 support 影像做比對，這樣既耗時又耗記憶體，所以 Prototypical Networks 的作者提出了一個想法，求出 support set 影像中每一個類別在 embedding space 的原形 (Prototypical)，也就是平均點，在計算 query 影像與原形的距離，將 query 影像分配給距離最近的類別，其分類概念與 LDA (Linear Discriminant Analysis) 一致

![](https://d3i71xaburhd42.cloudfront.net/81c2f2b347978b11ab6bf12aa86bb19a2042c818/2-Figure1-1.png)

注意到如果是 one-shot 任務的話，Prototypical Networks 等同於 Matching Networks





### Relation Network 2018

![](https://pic1.zhimg.com/v2-6c6bddb31b690745a8538c88b10608cf_720w.jpg?source=172ae18b)

注意到我們之前在計算 support set 影像與 query 影像的距離都是事前定義好的 (歐式距離等等)，Relation Networks 的作者利用 support set 影像和 query 影像的 embedding vector 做連接之後放入網路中學習一個 relation score，之後再對 relation score 做 softmax 即可完成分類

### In Practice

在實作部份我們會用簡單的 [olivetti](https://www.kaggle.com/datasets/imrandude/olivetti) 資料集

然而我推薦大家可以練習用 Omniglot 資料集，這是論文中常用的資料集

實作 colab notebook 連結 [Siamese](https://colab.research.google.com/github/DongDong-Zoez/pytorchAI/blob/main/FSL/iamese.ipynb)